In [1]:
import pandas as pd
import numpy as np
import math
from datetime import datetime
pd.set_option('display.max_columns', None)

Initial data cleaning and preliminary data exploration 

In [2]:
df = pd.read_csv('otomoto_scraping_2023-02-21.csv')

In [3]:
df.head()

,link,Oferta od,Kategoria,Generacja,Rok produkcji,Przebieg,Pojemność skokowa,Rodzaj paliwa,Moc,Skrzynia biegów,Napęd,Spalanie W Mieście,Typ nadwozia,Liczba drzwi,Liczba miejsc,Kolor,Kraj pochodzenia,Zarejestrowany w Polsce,Bezwypadkowy,Stan,Cena
0,https://www.otomoto.pl/oferta/volkswagen-golf-...,Osoby prywatnej,Osobowe,VII (2012-2020),2015,112 000 km,1 395 cm3,Benzyna,125 KM,Manualna,Na przednie koła,6 l/100km,Kompakt,5.0,5.0,Szary,Polska,Tak,Tak,Używane,56 400
1,https://www.otomoto.pl/oferta/volkswagen-golf-...,Osoby prywatnej,Osobowe,III (1991-1998),1997,230 000 km,1 781 cm3,Benzyna,75 KM,Manualna,Na przednie koła,11 l/100km,Kompakt,2.0,5.0,Czarny,NaN,Tak,Tak,Używane,3 500
2,https://www.otomoto.pl/oferta/volkswagen-golf-...,Osoby prywatnej,Osobowe,V (2003-2009),2008,269 000 km,1 390 cm3,Benzyna,122 KM,Manualna,NaN,NaN,Kombi,5.0,5.0,Szary,Holandia,NaN,Tak,Używane,16 500
3,https://www.otomoto.pl/oferta/volkswagen-golf-...,Firmy,Osobowe,VIII (2020-),2022,NaN,1 984 cm3,Benzyna,NaN,Automatyczna,4x4 (stały),NaN,Kompakt,NaN,NaN,Biały,Polska,NaN,Tak,Nowe,249 000
4,https://www.otomoto.pl/oferta/volkswagen-golf-...,Firmy,Osobowe,VIII (2020-),2022,NaN,1 984 cm3,Benzyna,NaN,Automatyczna,4x4 (stały),NaN,Kompakt,NaN,NaN,Czarny,Polska,NaN,Tak,Nowe,251 700


In [4]:
df.describe(include='all')

,link,Oferta od,Kategoria,Generacja,Rok produkcji,Przebieg,Pojemność skokowa,Rodzaj paliwa,Moc,Skrzynia biegów,Napęd,Spalanie W Mieście,Typ nadwozia,Liczba drzwi,Liczba miejsc,Kolor,Kraj pochodzenia,Zarejestrowany w Polsce,Bezwypadkowy,Stan,Cena
count,1794,1794,1794,1769,1794.000000,1778,1781,1794,1789,1791,1594,1105,1794,1790.000000,1752.000000,1794,1438,1082,1794,1794,1794
unique,1790,2,1,8,NaN,1079,29,6,49,2,5,54,8,NaN,NaN,17,15,1,1,2,742
top,https://www.otomoto.pl/oferta/volkswagen-golf-...,Firmy,Osobowe,VII (2012-2020),NaN,1 km,1 598 cm3,Benzyna,105 KM,Manualna,Na przednie koła,6 l/100km,Kompakt,NaN,NaN,Czarny,Polska,Tak,Tak,Używane,25 900
freq,2,932,1794,835,NaN,45,437,1001,287,1380,1505,223,1067,NaN,NaN,427,696,1082,1794,1677,24
mean,NaN,NaN,NaN,NaN,2013.056299,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.841341,4.990297,NaN,NaN,NaN,NaN,NaN,NaN
std,NaN,NaN,NaN,NaN,6.617226,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.558956,0.128327,NaN,NaN,NaN,NaN,NaN,NaN
min,NaN,NaN,NaN,NaN,1977.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.000000,2.000000,NaN,NaN,NaN,NaN,NaN,NaN
25%,NaN,NaN,NaN,NaN,2009.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.000000,5.000000,NaN,NaN,NaN,NaN,NaN,NaN
50%,NaN,NaN,NaN,NaN,2014.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.000000,5.000000,NaN,NaN,NaN,NaN,NaN,NaN
75%,NaN,NaN,NaN,NaN,2018.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.000000,5.000000,NaN,NaN,NaN,NaN,NaN,NaN


1. The column 'links' has to be checked for duplicates
2. Columns that provide little use for analysis will have to be dropped
3. Numeric values need to be converted to numeric
4. NaN values need to be filled with appropriate values


In [5]:
#First we investigate the duplicated links
df_links = pd.DataFrame(df['link'].value_counts()).reset_index()
duplicate_links = df_links['index'][df_links['link']==2].tolist()
#identical duplicates
df[df['link'].isin(duplicate_links)]

,link,Oferta od,Kategoria,Generacja,Rok produkcji,Przebieg,Pojemność skokowa,Rodzaj paliwa,Moc,Skrzynia biegów,Napęd,Spalanie W Mieście,Typ nadwozia,Liczba drzwi,Liczba miejsc,Kolor,Kraj pochodzenia,Zarejestrowany w Polsce,Bezwypadkowy,Stan,Cena
383,https://www.otomoto.pl/oferta/volkswagen-golf-...,Osoby prywatnej,Osobowe,VII (2012-2020),2015,111 300 km,1 395 cm3,Benzyna,122 KM,Automatyczna,Na przednie koła,6 l/100km,Kompakt,5.0,5.0,Czarny,Polska,NaN,Tak,Używane,60 885
384,https://www.otomoto.pl/oferta/volkswagen-golf-...,Osoby prywatnej,Osobowe,VII (2012-2020),2015,111 300 km,1 395 cm3,Benzyna,122 KM,Automatyczna,Na przednie koła,6 l/100km,Kompakt,5.0,5.0,Czarny,Polska,NaN,Tak,Używane,60 885
543,https://www.otomoto.pl/oferta/volkswagen-golf-...,Firmy,Osobowe,VII (2012-2020),2017,111 600 km,1 400 cm3,Benzyna,125 KM,Manualna,Na przednie koła,NaN,Kompakt,5.0,5.0,Czarny,Polska,Tak,Tak,Używane,67 600
544,https://www.otomoto.pl/oferta/volkswagen-golf-...,Firmy,Osobowe,VII (2012-2020),2017,111 600 km,1 400 cm3,Benzyna,125 KM,Manualna,Na przednie koła,NaN,Kompakt,5.0,5.0,Czarny,Polska,Tak,Tak,Używane,67 600
767,https://www.otomoto.pl/oferta/volkswagen-golf-...,Firmy,Osobowe,VII (2012-2020),2019,67 300 km,1 598 cm3,Diesel,116 KM,Manualna,Na przednie koła,NaN,Kombi,5.0,5.0,Biały,Polska,Tak,Tak,Używane,84 900
768,https://www.otomoto.pl/oferta/volkswagen-golf-...,Firmy,Osobowe,VII (2012-2020),2019,67 300 km,1 598 cm3,Diesel,116 KM,Manualna,Na przednie koła,NaN,Kombi,5.0,5.0,Biały,Polska,Tak,Tak,Używane,84 900
1087,https://www.otomoto.pl/oferta/volkswagen-golf-...,Firmy,Osobowe,IV (1997-2006),1999,98 000 km,1 390 cm3,Benzyna,75 KM,Manualna,Na przednie koła,8 l/100km,Auta miejskie,3.0,5.0,Czerwony,Niemcy,NaN,Tak,Używane,2 850
1088,https://www.otomoto.pl/oferta/volkswagen-golf-...,Firmy,Osobowe,IV (1997-2006),1999,98 000 km,1 390 cm3,Benzyna,75 KM,Manualna,Na przednie koła,8 l/100km,Auta miejskie,3.0,5.0,Czerwony,Niemcy,NaN,Tak,Używane,2 850


In [6]:
#Those look like regular duplicates so let's get rid of them
df.drop_duplicates(subset=['link'], inplace=True)
#check
df[df['link'].isin(duplicate_links)]

,link,Oferta od,Kategoria,Generacja,Rok produkcji,Przebieg,Pojemność skokowa,Rodzaj paliwa,Moc,Skrzynia biegów,Napęd,Spalanie W Mieście,Typ nadwozia,Liczba drzwi,Liczba miejsc,Kolor,Kraj pochodzenia,Zarejestrowany w Polsce,Bezwypadkowy,Stan,Cena
383,https://www.otomoto.pl/oferta/volkswagen-golf-...,Osoby prywatnej,Osobowe,VII (2012-2020),2015,111 300 km,1 395 cm3,Benzyna,122 KM,Automatyczna,Na przednie koła,6 l/100km,Kompakt,5.0,5.0,Czarny,Polska,NaN,Tak,Używane,60 885
543,https://www.otomoto.pl/oferta/volkswagen-golf-...,Firmy,Osobowe,VII (2012-2020),2017,111 600 km,1 400 cm3,Benzyna,125 KM,Manualna,Na przednie koła,NaN,Kompakt,5.0,5.0,Czarny,Polska,Tak,Tak,Używane,67 600
767,https://www.otomoto.pl/oferta/volkswagen-golf-...,Firmy,Osobowe,VII (2012-2020),2019,67 300 km,1 598 cm3,Diesel,116 KM,Manualna,Na przednie koła,NaN,Kombi,5.0,5.0,Biały,Polska,Tak,Tak,Używane,84 900
1087,https://www.otomoto.pl/oferta/volkswagen-golf-...,Firmy,Osobowe,IV (1997-2006),1999,98 000 km,1 390 cm3,Benzyna,75 KM,Manualna,Na przednie koła,8 l/100km,Auta miejskie,3.0,5.0,Czerwony,Niemcy,NaN,Tak,Używane,2 850


In [7]:
#Next let's convert numeric data to int
#cena
df['Cena'] = df['Cena'].str.replace(' ', '')
df['Cena'] = df['Cena'].astype('int64')
df['Cena'].describe()

count      1790.000000
mean      55505.897207
std       46659.580120
min        2300.000000
25%       22100.000000
50%       45900.000000
75%       69900.000000
max      280080.000000
Name: Cena, dtype: float64

In [8]:
df['Przebieg'] = df['Przebieg'].str.replace(' km', '')
df['Przebieg'] = df['Przebieg'].str.replace(' ', '')

In [9]:
#investigate NaN values
df[['Rok produkcji']][df['Przebieg'].isna()].value_counts()

Rok produkcji
2023             12
2022              4
dtype: int64

In [10]:
#all of those cars are brand new so let's set the values to 0 and cast as numeric
df['Przebieg'] = pd.to_numeric(df['Przebieg'].fillna(0))
#df['Przebieg'] = pd.to_numeric(df['Przebieg'])
df['Przebieg'].describe()

count      1790.000000
mean     157260.482123
std       83671.298916
min           0.000000
25%      100594.000000
50%      167415.000000
75%      212754.500000
max      435000.000000
Name: Przebieg, dtype: float64

In [11]:
#Pojemność skokowa - same treatment
df['Pojemność skokowa'] = df['Pojemność skokowa'].str.replace(' cm3', '')
df['Pojemność skokowa'] = df['Pojemność skokowa'].str.replace(' ', '')
df['Pojemność skokowa'] = pd.to_numeric(df['Pojemność skokowa'])

In [12]:
#checking NaNs
df[df['Pojemność skokowa'].isna()]
#NaNs for electric cars

,link,Oferta od,Kategoria,Generacja,Rok produkcji,Przebieg,Pojemność skokowa,Rodzaj paliwa,Moc,Skrzynia biegów,Napęd,Spalanie W Mieście,Typ nadwozia,Liczba drzwi,Liczba miejsc,Kolor,Kraj pochodzenia,Zarejestrowany w Polsce,Bezwypadkowy,Stan,Cena
151,https://www.otomoto.pl/oferta/volkswagen-golf-...,Osoby prywatnej,Osobowe,VII (2012-2020),2019,22000,NaN,Elektryczny,136 KM,Automatyczna,Na przednie koła,NaN,Kompakt,5.0,5.0,Biały,Niemcy,Tak,Tak,Używane,125999
177,https://www.otomoto.pl/oferta/volkswagen-golf-...,Osoby prywatnej,Osobowe,VII (2012-2020),2019,42000,NaN,Elektryczny,136 KM,Automatyczna,Na przednie koła,NaN,Kompakt,5.0,5.0,Srebrny,Polska,NaN,Tak,Używane,119999
185,https://www.otomoto.pl/oferta/volkswagen-golf-...,Firmy,Osobowe,VII (2012-2020),2017,161627,NaN,Elektryczny,136 KM,Automatyczna,Na przednie koła,NaN,Kompakt,5.0,5.0,Biały,NaN,Tak,Tak,Używane,97170
318,https://www.otomoto.pl/oferta/volkswagen-golf-...,Osoby prywatnej,Osobowe,VII (2012-2020),2016,70000,NaN,Elektryczny,115 KM,Automatyczna,Na przednie koła,NaN,Kompakt,5.0,5.0,Czarny,Norwegia,Tak,Tak,Używane,85900
458,https://www.otomoto.pl/oferta/volkswagen-golf-...,Firmy,Osobowe,VII (2012-2020),2019,47301,NaN,Elektryczny,136 KM,NaN,Na przednie koła,NaN,Kompakt,5.0,5.0,Biały,NaN,Tak,Tak,Używane,117000
460,https://www.otomoto.pl/oferta/volkswagen-golf-...,Firmy,Osobowe,VII (2012-2020),2020,60851,NaN,Elektryczny,136 KM,NaN,Na przednie koła,NaN,Kompakt,5.0,5.0,Szary,NaN,Tak,Tak,Używane,115000
694,https://www.otomoto.pl/oferta/volkswagen-golf-...,Osoby prywatnej,Osobowe,VII (2012-2020),2018,64000,NaN,Elektryczny,136 KM,Automatyczna,Na przednie koła,NaN,Kompakt,5.0,5.0,Beżowy,Norwegia,NaN,Tak,Używane,99000
913,https://www.otomoto.pl/oferta/volkswagen-golf-...,Firmy,Osobowe,VII (2012-2020),2015,71500,NaN,Elektryczny,115 KM,Automatyczna,Na przednie koła,NaN,Kompakt,5.0,5.0,Biały,NaN,Tak,Tak,Używane,84900
917,https://www.otomoto.pl/oferta/volkswagen-golf-...,Firmy,Osobowe,VII (2012-2020),2015,77000,NaN,Elektryczny,115 KM,Automatyczna,Na przednie koła,NaN,Kompakt,5.0,5.0,Czarny,NaN,NaN,Tak,Używane,70900
975,https://www.otomoto.pl/oferta/volkswagen-golf-...,Osoby prywatnej,Osobowe,VII (2012-2020),2018,72000,NaN,Elektryczny,136 KM,Automatyczna,NaN,NaN,Kompakt,5.0,NaN,Czarny,Norwegia,NaN,Tak,Używane,109000


In [13]:
#Checking counts of fuel type
df['Rodzaj paliwa'].value_counts()

Benzyna        998
Diesel         758
Benzyna+LPG     16
Elektryczny     13
Hybryda          4
Benzyna+CNG      1
Name: Rodzaj paliwa, dtype: int64

In [14]:
#because there are very few examples of fuel types other than Diesel and Benzyna we drop them
df.drop(df[~(df['Rodzaj paliwa'].isin(['Benzyna', 'Diesel']))].index, inplace=True)
df['Rodzaj paliwa'].value_counts()

Benzyna    998
Diesel     758
Name: Rodzaj paliwa, dtype: int64

In [15]:
#Repeat for Moc
df['Moc'] = df['Moc'].str.replace(' KM', '')
df['Moc'] = pd.to_numeric(df['Moc'])
df[df['Moc'].isna()]

,link,Oferta od,Kategoria,Generacja,Rok produkcji,Przebieg,Pojemność skokowa,Rodzaj paliwa,Moc,Skrzynia biegów,Napęd,Spalanie W Mieście,Typ nadwozia,Liczba drzwi,Liczba miejsc,Kolor,Kraj pochodzenia,Zarejestrowany w Polsce,Bezwypadkowy,Stan,Cena
3,https://www.otomoto.pl/oferta/volkswagen-golf-...,Firmy,Osobowe,VIII (2020-),2022,0,1984.0,Benzyna,NaN,Automatyczna,4x4 (stały),NaN,Kompakt,NaN,NaN,Biały,Polska,NaN,Tak,Nowe,249000
4,https://www.otomoto.pl/oferta/volkswagen-golf-...,Firmy,Osobowe,VIII (2020-),2022,0,1984.0,Benzyna,NaN,Automatyczna,4x4 (stały),NaN,Kompakt,NaN,NaN,Czarny,Polska,NaN,Tak,Nowe,251700
25,https://www.otomoto.pl/oferta/volkswagen-golf-...,Firmy,Osobowe,VIII (2020-),2022,0,1984.0,Benzyna,NaN,Automatyczna,4x4 (stały),NaN,Kompakt,NaN,NaN,Niebieski,Polska,NaN,Tak,Nowe,250380
1585,https://www.otomoto.pl/oferta/volkswagen-golf-...,Firmy,Osobowe,NaN,2005,190000,1896.0,Diesel,NaN,Manualna,Na przednie koła,NaN,Kompakt,5.0,5.0,Srebrny,Belgia,NaN,Tak,Używane,22500
1586,https://www.otomoto.pl/oferta/volkswagen-golf-...,Firmy,Osobowe,NaN,2009,168000,1998.0,Diesel,NaN,Manualna,Na przednie koła,NaN,Kompakt,5.0,5.0,Srebrny,Belgia,NaN,Tak,Używane,34900


In [16]:
df['Moc'] = df['Moc'].fillna(df.groupby(['Rok produkcji', 'Pojemność skokowa', 'Rodzaj paliwa'])['Moc'].transform('mean'))
#still one more to fill, we'll use overall mean
df[df['Moc'].isna()]

,link,Oferta od,Kategoria,Generacja,Rok produkcji,Przebieg,Pojemność skokowa,Rodzaj paliwa,Moc,Skrzynia biegów,Napęd,Spalanie W Mieście,Typ nadwozia,Liczba drzwi,Liczba miejsc,Kolor,Kraj pochodzenia,Zarejestrowany w Polsce,Bezwypadkowy,Stan,Cena
1586,https://www.otomoto.pl/oferta/volkswagen-golf-...,Firmy,Osobowe,NaN,2009,168000,1998.0,Diesel,NaN,Manualna,Na przednie koła,NaN,Kompakt,5.0,5.0,Srebrny,Belgia,NaN,Tak,Używane,34900


In [17]:
df['Moc'] = df['Moc'].fillna(df['Moc'].mean())
#finally it's empty
df[df['Moc'].isna()]

,link,Oferta od,Kategoria,Generacja,Rok produkcji,Przebieg,Pojemność skokowa,Rodzaj paliwa,Moc,Skrzynia biegów,Napęd,Spalanie W Mieście,Typ nadwozia,Liczba drzwi,Liczba miejsc,Kolor,Kraj pochodzenia,Zarejestrowany w Polsce,Bezwypadkowy,Stan,Cena


In [18]:
#verifying if the mean was calculated correctly
print(df['Moc'].loc[(df['Rok produkcji']==2022) & (df['Pojemność skokowa']==1984) & df['Moc'].notnull() & (df['Rodzaj paliwa']=='Benzyna')].mean())
print(df['Moc'].loc[25])

304.9
304.9


In [19]:
df['Skrzynia biegów'].value_counts()

Manualna        1361
Automatyczna     395
Name: Skrzynia biegów, dtype: int64

In [20]:
#too few examples of drives other than front wheels
df['Napęd'].value_counts()

Na przednie koła                 1472
4x4 (dołączany automatycznie)      58
4x4 (stały)                        18
4x4 (dołączany ręcznie)             8
Na tylne koła                       5
Name: Napęd, dtype: int64

In [21]:
df.drop(df[~(df['Napęd'].isin(['Na przednie koła']))].index, inplace=True)
df['Napęd'].value_counts()

Na przednie koła    1472
Name: Napęd, dtype: int64

In [22]:
#convert to int and fillna based on Pojemność Skokowa and Moc
df[df['Spalanie W Mieście'].isna()]

,link,Oferta od,Kategoria,Generacja,Rok produkcji,Przebieg,Pojemność skokowa,Rodzaj paliwa,Moc,Skrzynia biegów,Napęd,Spalanie W Mieście,Typ nadwozia,Liczba drzwi,Liczba miejsc,Kolor,Kraj pochodzenia,Zarejestrowany w Polsce,Bezwypadkowy,Stan,Cena
21,https://www.otomoto.pl/oferta/volkswagen-golf-...,Firmy,Osobowe,VIII (2020-),2023,8,1498.0,Benzyna,150.0,Automatyczna,Na przednie koła,NaN,Kompakt,5.0,5.0,Żółty,Polska,NaN,Tak,Nowe,169900
28,https://www.otomoto.pl/oferta/volkswagen-golf-...,Firmy,Osobowe,VII (2012-2020),2014,189736,1598.0,Diesel,105.0,Manualna,Na przednie koła,NaN,Kombi,5.0,5.0,Srebrny,Niemcy,Tak,Tak,Używane,39999
29,https://www.otomoto.pl/oferta/volkswagen-golf-...,Firmy,Osobowe,VII (2012-2020),2019,74674,1984.0,Benzyna,245.0,Automatyczna,Na przednie koła,NaN,Kompakt,5.0,5.0,Biały,Polska,Tak,Tak,Używane,114900
30,https://www.otomoto.pl/oferta/volkswagen-golf-...,Firmy,Osobowe,VIII (2020-),2023,5,1984.0,Benzyna,245.0,Automatyczna,Na przednie koła,NaN,Kompakt,5.0,5.0,Biały,Polska,NaN,Tak,Nowe,190900
32,https://www.otomoto.pl/oferta/volkswagen-golf-...,Firmy,Osobowe,VII (2012-2020),2018,49494,1498.0,Benzyna,150.0,Automatyczna,Na przednie koła,NaN,Kombi,5.0,5.0,Czarny,Polska,Tak,Tak,Używane,99900
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1758,https://www.otomoto.pl/oferta/volkswagen-golf-...,Osoby prywatnej,Osobowe,NaN,2018,50000,999.0,Benzyna,110.0,Manualna,Na przednie koła,NaN,Kombi,5.0,NaN,Szary,Polska,Tak,Tak,Używane,56900
1760,https://www.otomoto.pl/oferta/volkswagen-golf-...,Firmy,Osobowe,VII (2012-2020),2018,181950,1598.0,Diesel,115.0,Manualna,Na przednie koła,NaN,Kombi,5.0,5.0,Srebrny,Polska,Tak,Tak,Używane,47900
1763,https://www.otomoto.pl/oferta/volkswagen-golf-...,Osoby prywatnej,Osobowe,VII (2012-2020),2016,144000,1600.0,Diesel,110.0,Manualna,Na przednie koła,NaN,Kombi,5.0,5.0,Srebrny,Niemcy,Tak,Tak,Używane,56900
1772,https://www.otomoto.pl/oferta/volkswagen-golf-...,Firmy,Osobowe,VIII (2020-),2020,44216,1498.0,Benzyna,130.0,Manualna,Na przednie koła,NaN,Kompakt,5.0,5.0,Szary,Polska,Tak,Tak,Używane,105900


In [23]:
df['Spalanie W Mieście'] = df['Spalanie W Mieście'].str.replace(' l/100km', '')
df['Spalanie W Mieście'] = df['Spalanie W Mieście'].str.replace(',', '.')
df['Spalanie W Mieście'] = pd.to_numeric(df['Spalanie W Mieście'])

In [24]:
#the values have to be rounded
df2 = df.copy()
df2['Pojemność skokowa'] = df2['Pojemność skokowa'].round(-3)
df2['Moc'] = df2['Moc'].round(-2)

In [25]:
#filling missing values with rough means for groups
df['Spalanie W Mieście'] = df['Spalanie W Mieście'].fillna(df2.groupby(['Pojemność skokowa', 'Moc'])['Spalanie W Mieście'].transform('mean'))

In [26]:
#one more left
print(df['Spalanie W Mieście'][df['Spalanie W Mieście'].isna()])
df['Spalanie W Mieście'] = df['Spalanie W Mieście'].fillna(df['Spalanie W Mieście'].mean())
print(df['Spalanie W Mieście'][df['Spalanie W Mieście'].isna()])

654   NaN
Name: Spalanie W Mieście, dtype: float64
Series([], Name: Spalanie W Mieście, dtype: float64)


In [27]:
df['Typ nadwozia'].value_counts()

Kompakt          867
Kombi            551
Auta miejskie     36
Auta małe          9
Minivan            4
Kabriolet          2
Sedan              2
Coupe              1
Name: Typ nadwozia, dtype: int64

In [28]:
#set 'Auta miejskie' and 'Auta małe' as 'Kompakt', drop the remaining categories with <10 examples
df.loc[(df['Typ nadwozia']=='Auta miejskie') | (df['Typ nadwozia']=='Auta małe'), 'Typ nadwozia'] = 'Kompakt'
df.drop(df[~(df['Typ nadwozia'].isin(['Kompakt', 'Kombi']))].index, inplace=True)
df['Typ nadwozia'].value_counts()

Kompakt    912
Kombi      551
Name: Typ nadwozia, dtype: int64

In [29]:
#set NaN and countries other than DE and PL as 'Other'
df['Kraj pochodzenia'].value_counts(dropna=False)

Polska               568
Niemcy               529
NaN                  268
Belgia                23
Francja               20
Holandia              19
Włochy                13
Austria                6
Szwecja                6
Dania                  4
Szwajcaria             3
Kanada                 1
Stany Zjednoczone      1
Luksemburg             1
Hiszpania              1
Name: Kraj pochodzenia, dtype: int64

In [30]:
df.loc[~df['Kraj pochodzenia'].isin(['Polska', 'Niemcy']), 'Kraj pochodzenia'] = 'Inne'
df['Kraj pochodzenia'].value_counts(dropna=False)

Polska    568
Niemcy    529
Inne      366
Name: Kraj pochodzenia, dtype: int64

In [31]:
df['Zarejestrowany w Polsce'].value_counts(dropna=False)
df.loc[(df['Zarejestrowany w Polsce'].isna()),'Zarejestrowany w Polsce'] = 'Nie'

In [32]:
df.describe(include='all')

,link,Oferta od,Kategoria,Generacja,Rok produkcji,Przebieg,Pojemność skokowa,Rodzaj paliwa,Moc,Skrzynia biegów,Napęd,Spalanie W Mieście,Typ nadwozia,Liczba drzwi,Liczba miejsc,Kolor,Kraj pochodzenia,Zarejestrowany w Polsce,Bezwypadkowy,Stan,Cena
count,1463,1463,1463,1445,1463.000000,1463.000000,1463.000000,1463,1463.000000,1463,1463,1463.000000,1463,1462.000000,1445.000000,1463,1463,1463,1463,1463,1463.000000
unique,1463,2,1,8,NaN,NaN,NaN,2,NaN,2,1,NaN,2,NaN,NaN,17,3,2,1,2,NaN
top,https://www.otomoto.pl/oferta/volkswagen-golf-...,Firmy,Osobowe,VII (2012-2020),NaN,NaN,NaN,Benzyna,NaN,Manualna,Na przednie koła,NaN,Kompakt,NaN,NaN,Czarny,Polska,Tak,Tak,Używane,NaN
freq,1,805,1463,681,NaN,NaN,NaN,815,NaN,1175,1463,NaN,912,NaN,NaN,351,568,878,1463,1374,NaN
mean,NaN,NaN,NaN,NaN,2013.088859,158328.269993,1587.277512,NaN,122.884077,NaN,NaN,6.595164,NaN,4.855677,4.996540,NaN,NaN,NaN,NaN,NaN,51484.298018
std,NaN,NaN,NaN,NaN,6.471153,82326.899191,262.245493,NaN,35.980791,NaN,NaN,1.402192,NaN,0.526211,0.078871,NaN,NaN,NaN,NaN,NaN,37477.930638
min,NaN,NaN,NaN,NaN,1982.000000,0.000000,999.000000,NaN,50.000000,NaN,NaN,3.000000,NaN,2.000000,3.000000,NaN,NaN,NaN,NaN,NaN,2300.000000
25%,NaN,NaN,NaN,NaN,2009.000000,104000.000000,1395.000000,NaN,105.000000,NaN,NaN,6.000000,NaN,5.000000,5.000000,NaN,NaN,NaN,NaN,NaN,22500.000000
50%,NaN,NaN,NaN,NaN,2014.000000,167398.000000,1598.000000,NaN,115.000000,NaN,NaN,6.164663,NaN,5.000000,5.000000,NaN,NaN,NaN,NaN,NaN,45900.000000
75%,NaN,NaN,NaN,NaN,2018.000000,212552.000000,1896.000000,NaN,140.000000,NaN,NaN,7.190940,NaN,5.000000,5.000000,NaN,NaN,NaN,NaN,NaN,67900.000000


In [33]:
today = datetime.today().strftime('%Y-%m-%d')
file_name = f'data_clean_{today}.csv'
df.to_csv(file_name, index=False)

In [34]:
#get rid of columns we don't need for analysis
df.drop(columns=['link', 'Kategoria', 'Generacja', 'Napęd', 'Liczba drzwi', 'Liczba miejsc', 'Typ nadwozia', 'Kolor', 'Bezwypadkowy', 'Stan'], inplace=True)

In [35]:
df.head()

,Oferta od,Rok produkcji,Przebieg,Pojemność skokowa,Rodzaj paliwa,Moc,Skrzynia biegów,Spalanie W Mieście,Kraj pochodzenia,Zarejestrowany w Polsce,Cena
0,Osoby prywatnej,2015,112000,1395.0,Benzyna,125.0,Manualna,6.0,Polska,Tak,56400
1,Osoby prywatnej,1997,230000,1781.0,Benzyna,75.0,Manualna,11.0,Inne,Tak,3500
5,Osoby prywatnej,2007,126296,1390.0,Benzyna,140.0,Manualna,9.0,Niemcy,Nie,19900
6,Osoby prywatnej,2007,367800,1896.0,Diesel,105.0,Manualna,6.0,Niemcy,Tak,14000
7,Osoby prywatnej,2008,293000,1896.0,Diesel,105.0,Manualna,8.0,Polska,Tak,13500


In [36]:
df_ml = pd.get_dummies(df.reset_index(drop=True))
df_ml

,Rok produkcji,Przebieg,Pojemność skokowa,Moc,Spalanie W Mieście,Cena,Oferta od_Firmy,Oferta od_Osoby prywatnej,Rodzaj paliwa_Benzyna,Rodzaj paliwa_Diesel,Skrzynia biegów_Automatyczna,Skrzynia biegów_Manualna,Kraj pochodzenia_Inne,Kraj pochodzenia_Niemcy,Kraj pochodzenia_Polska,Zarejestrowany w Polsce_Nie,Zarejestrowany w Polsce_Tak
0,2015,112000,1395.0,125.0,6.000000,56400,0,1,1,0,0,1,0,0,1,0,1
1,1997,230000,1781.0,75.0,11.000000,3500,0,1,1,0,0,1,1,0,0,0,1
2,2007,126296,1390.0,140.0,9.000000,19900,0,1,1,0,0,1,0,1,0,1,0
3,2007,367800,1896.0,105.0,6.000000,14000,0,1,0,1,0,1,0,1,0,0,1
4,2008,293000,1896.0,105.0,8.000000,13500,0,1,0,1,0,1,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1458,1988,142961,1781.0,107.0,6.164663,49999,1,0,1,0,0,1,1,0,0,1,0
1459,2015,52109,1395.0,125.0,6.000000,66800,1,0,1,0,1,0,0,0,1,0,1
1460,2019,58000,1498.0,150.0,6.100000,85500,0,1,1,0,1,0,1,0,0,0,1
1461,2005,195670,1595.0,102.0,9.000000,17000,1,0,1,0,0,1,0,1,0,1,0


In [37]:
file_name = f'ML_data_{today}.csv'
df_ml.to_csv(file_name, index=False)

In [38]:
#checking scalers -- for later
from sklearn.preprocessing import StandardScaler, MinMaxScaler, MaxAbsScaler

In [39]:
scaler = StandardScaler()
scaler.fit(df_ml)
pd.DataFrame(scaler.transform(df_ml))

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,0.295433,-0.562928,-0.733447,0.058827,-0.424597,0.131207,-1.106076,1.106076,0.891679,-0.891679,-0.495082,0.495082,-0.577613,-0.752583,1.255271,-0.816264,0.816264
1,-2.487093,0.870873,0.738959,-1.331278,3.142467,-1.280773,-1.106076,1.106076,0.891679,-0.891679,-0.495082,0.495082,1.731262,-0.752583,-0.796641,-0.816264,0.816264
2,-0.941245,-0.389219,-0.752520,0.475859,1.715641,-0.843032,-1.106076,1.106076,0.891679,-0.891679,-0.495082,0.495082,-0.577613,1.328757,-0.796641,1.225094,-1.225094
3,-0.941245,2.545260,1.177630,-0.497215,-0.424597,-1.000512,-1.106076,1.106076,-1.121479,1.121479,-0.495082,0.495082,-0.577613,1.328757,-0.796641,-0.816264,0.816264
4,-0.786660,1.636376,1.177630,-0.497215,1.002228,-1.013858,-1.106076,1.106076,-1.121479,1.121479,-0.495082,0.495082,-0.577613,-0.752583,1.255271,-0.816264,0.816264
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1458,-3.878356,-0.186725,0.738959,-0.441611,-0.307124,-0.039645,0.904097,-0.904097,0.891679,-0.891679,-0.495082,0.495082,1.731262,-0.752583,-0.796641,1.225094,-1.225094
1459,0.295433,-1.290655,-0.733447,0.058827,-0.424597,0.408799,0.904097,-0.904097,0.891679,-0.891679,2.019867,-2.019867,-0.577613,-0.752583,1.255271,-0.816264,0.816264
1460,0.913773,-1.219074,-0.340551,0.753880,-0.353256,0.907930,-1.106076,1.106076,0.891679,-0.891679,2.019867,-2.019867,1.731262,-0.752583,-0.796641,-0.816264,0.816264
1461,-1.250415,0.453734,0.029458,-0.580621,1.715641,-0.920437,0.904097,-0.904097,0.891679,-0.891679,-0.495082,0.495082,-0.577613,1.328757,-0.796641,1.225094,-1.225094


In [40]:
scaler = MinMaxScaler()
scaler.fit(df_ml)
pd.DataFrame(scaler.transform(df_ml))

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,0.804878,0.257471,0.180822,0.279851,0.230769,0.266031,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0
1,0.365854,0.528736,0.357078,0.093284,0.615385,0.005901,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0
2,0.609756,0.290336,0.178539,0.335821,0.461538,0.086546,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
3,0.609756,0.845517,0.409589,0.205224,0.230769,0.057533,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
4,0.634146,0.673563,0.409589,0.205224,0.384615,0.055075,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1458,0.146341,0.328646,0.357078,0.212687,0.243436,0.234554,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0
1459,0.804878,0.119791,0.180822,0.279851,0.230769,0.317172,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0
1460,0.902439,0.133333,0.227854,0.373134,0.238462,0.409127,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0
1461,0.560976,0.449816,0.272146,0.194030,0.461538,0.072286,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0


In [41]:
scaler = MaxAbsScaler()
scaler.fit(df_ml)
pd.DataFrame(scaler.transform(df_ml))

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,0.996045,0.257471,0.437441,0.393082,0.375000,0.274239,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0
1,0.987148,0.528736,0.558482,0.235849,0.687500,0.017018,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0
2,0.992091,0.290336,0.435873,0.440252,0.562500,0.096762,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
3,0.992091,0.845517,0.594544,0.330189,0.375000,0.068074,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
4,0.992585,0.673563,0.594544,0.330189,0.500000,0.065642,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1458,0.982699,0.328646,0.558482,0.336478,0.385291,0.243115,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0
1459,0.996045,0.119791,0.437441,0.393082,0.375000,0.324808,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0
1460,0.998023,0.133333,0.469740,0.471698,0.381250,0.415735,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0
1461,0.991102,0.449816,0.500157,0.320755,0.562500,0.082661,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0


In [42]:
(df_ml['Oferta od_Firmy'] - df_ml['Oferta od_Firmy'].mean())/df_ml['Oferta od_Firmy'].std()

0      -1.105698
1      -1.105698
2      -1.105698
3      -1.105698
4      -1.105698
          ...   
1458    0.903788
1459    0.903788
1460   -1.105698
1461    0.903788
1462    0.903788
Name: Oferta od_Firmy, Length: 1463, dtype: float64